In [1]:
import numpy as np
import sys
import os
import pandas as pd
import timm
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import datasets, transforms
from torchvision.models import resnet50
from transformers import ViTModel

sys.path.append("ssl_library")
from src.pkg.embedder import Embedder
from src.pkg.wrappers import ViTWrapper

In [2]:
seed = 19
img_size = 224
batch_size = 16
normalise_mean = (0.485, 0.456, 0.406)  # ImageNet
normalise_std = (0.229, 0.224, 0.225)  # ImageNet

configuration_csv_path = "configs/tasks-configuration-new.csv"

In [3]:
torch.manual_seed(seed)

In [4]:
assert torch.cuda.is_available()
n_devices = torch.cuda.device_count()
for i in range(0, n_devices):
    print(torch.cuda.get_device_name(i))

device = torch.device("cuda")

NVIDIA GeForce GTX 960


In [5]:
def load_model(model_architecture, checkpoint_path):
    model = None

    if model_architecture == "resnet50":
        print(f"Implement!")
    elif model_architecture == "swin_t":
        print(f"Implement!")
    elif model_architecture == "vit_b16":
        print(f"Implement!")
    elif model_architecture == "ViT_T16":
        if "vit_t16_v1" in checkpoint_path:
            model = Embedder.load_pretrained("imagenet_vit_tiny")
        elif "vit_t16_v2" in checkpoint_path:
            model = Embedder.load_pretrained("vit_tiny_random")
        elif "vit_t16_v3" in checkpoint_path:
            # NOTE: VisionTransformer from timm neds to be wrapped to get intermediate results
            model = timm.create_model("vit_tiny_patch16_224", pretrained=False)
            model = ViTWrapper(model)
        model.head = nn.Sequential()
    else:
        print(f"Unknown model architecture: {model_architecture}")
        return None

    assert model != None
    print(f"Loading {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))
    model.load_state_dict(checkpoint, strict=True)

    for param in model.parameters():
        param.requires_grad = False

    return model

In [6]:
class ImageFolderExtented(datasets.ImageFolder):
    # NOTE: ImageFolder uses pil_loader as default, which executes Image.convert("RGB") implicitly
    def __getitem__(self, index: int):
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target, os.path.basename(path)


def load_dataloader(data_dir):
    transform = transforms.Compose(
        [
            transforms.Resize((256, 256)),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize(normalise_mean, normalise_std),
        ]
    )
    ds_full = ImageFolderExtented(data_dir, transform=transform)

    dl_full = data.DataLoader(
        ds_full,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
    )
    return dl_full

In [7]:
def calculate_features(model, dl_full):
    model = model.to(device)
    model.eval()
    target_list = []
    features_list = []
    filename_list = []

    for i, (images, targets, filenames) in enumerate(dl_full):
        # print(f"Batch {i}")
        filename_list.extend(filenames)

        numpy_targets = targets.numpy()
        target_list.append(numpy_targets)

        outputs = numpy_targets
        images = images.to(device)

        # NOTE: VitWrapper automatically returns results of last for blocks
        outputs = model(images)

        # numpy_outputs = None
        # if "BaseModelOutputWithPooling" in str(type(outputs)):
        #     preferred_shape = [numpy_targets.shape[0], -1] # Shape must be 2D
        #     numpy_outputs = outputs.last_hidden_state.cpu().numpy().reshape(preferred_shape)
        # else:
        numpy_outputs = outputs.cpu().numpy()
        features_list.append(numpy_outputs)

    print(f"Number of batches: {len(target_list)}")
    np_features = np.concatenate(features_list)
    np_target = np.concatenate(target_list)  # .reshape((np_features.shape[0], 1))
    df_full = pd.DataFrame(np_features)
    df_full["target_num"] = pd.Series(np_target)
    df_full["filename"] = pd.Series(filename_list)
    return df_full

In [8]:
def split_and_save_dataframe(df_features, csv_path):
    df_split = pd.read_csv(os.path.join(dataset_path, "split.csv"), index_col=0)
    assert (
        df_split["target_code"].unique().size == df_features["target_num"].unique().size
    )
    assert df_split["filename"].unique().size == df_features["filename"].unique().size
    df_split["filename"] = df_split["filename"].apply(lambda x: os.path.splitext(x)[0])
    df_features["filename"] = df_features["filename"].apply(
        lambda x: os.path.splitext(x)[0]
    )
    df_merged = pd.merge(df_features, df_split, on="filename")
    groups = df_merged.groupby(["target_code", "target_num"])
    assert groups["set"].count().size == df_split["target_code"].unique().size

    df_merged.drop(columns=["filename", "target_code"], inplace=True)
    cols = df_merged.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    df_merged = df_merged[cols]
    df_merged.to_csv(csv_path)
    print(f"Csv file saved: {feature_path}")

In [10]:
df_config = pd.read_csv(configuration_csv_path)
counter = 0

df_config

for index, row in df_config.iterrows():
    dataset_path = str(row["dataset_path"])
    architecture = str(row["architecture"])
    weigths_path = str(row["weigths_path"])
    feature_path = str(row["feature_path"])

    target_filename = os.path.basename(feature_path)
    name_parts = target_filename.split(".")
    assert len(name_parts) == 2
    assert name_parts[1] == "csv"
    name_parts = name_parts[0].split("-")
    assert len(name_parts) == 3
    assert name_parts[0].lower() in dataset_path.replace("-", "_").lower()
    assert name_parts[1] == architecture
    assert name_parts[2] in weigths_path

    if os.path.exists(feature_path):
        print(f"Csv file already exists: {feature_path}")
        counter += 1
    else:
        if 0 < counter:
            print(f"Skipped {counter} existing files")
            counter = 0
        model = load_model(architecture, weigths_path)
        dataloader = load_dataloader(dataset_path)
        df_features = calculate_features(model, dataloader)
        split_and_save_dataframe(df_features, feature_path)

Csv file already exists: ../datasets/intermediate-features/PAD_UFES_20-ViT_T16-Derma.csv
Csv file already exists: ../datasets/intermediate-features/PAD_UFES_20-ViT_T16-ImageNet_1k_SL_WinKawaks.csv
Csv file already exists: ../datasets/intermediate-features/PAD_UFES_20-ViT_T16-ImageNet_1k_SSL_Dino.csv
Csv file already exists: ../datasets/intermediate-features/PAD_UFES_20-ViT_T16-ImageNet_AugReg.csv
Csv file already exists: ../datasets/intermediate-features/PAD_UFES_20-ViT_T16-Plant.csv
Csv file already exists: ../datasets/intermediate-features/PAD_UFES_20-ViT_T16-Random.csv
Csv file already exists: ../datasets/intermediate-features/DDI-ViT_T16-Derma.csv
Csv file already exists: ../datasets/intermediate-features/DDI-ViT_T16-ImageNet_1k_SL_WinKawaks.csv
Csv file already exists: ../datasets/intermediate-features/DDI-ViT_T16-ImageNet_1k_SSL_Dino.csv
Csv file already exists: ../datasets/intermediate-features/DDI-ViT_T16-ImageNet_AugReg.csv
Csv file already exists: ../datasets/intermediate-fea

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading ../model_weights/vit_t16_v1/ViT_T16-ImageNet_1k_SL_WinKawaks_headless.pth


c:\workspace\thesis\.venv\lib\site-packages\transformers\models\vit\modeling_vit.py:253: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(


Number of batches: 721
Csv file saved: ../datasets/intermediate-features/HAM10000-ViT_T16-ImageNet_1k_SL_WinKawaks.csv
Loading ../model_weights/vit_t16_v2/ViT_T16-ImageNet_1k_SSL_Dino_headless.pth
Number of batches: 721
Csv file saved: ../datasets/intermediate-features/HAM10000-ViT_T16-ImageNet_1k_SSL_Dino.csv
Loading ../model_weights/vit_t16_v3/ViT_T16-ImageNet_AugReg_headless.pth
Number of batches: 721
Csv file saved: ../datasets/intermediate-features/HAM10000-ViT_T16-ImageNet_AugReg.csv
Loading ../model_weights/vit_t16_v2/ViT_T16-Plant_SSL_Dino_headless.pth
Number of batches: 721
Csv file saved: ../datasets/intermediate-features/HAM10000-ViT_T16-Plant.csv
Loading ../model_weights/vit_t16_v2/ViT_T16-Random_headless.pth
Number of batches: 721
Csv file saved: ../datasets/intermediate-features/HAM10000-ViT_T16-Random.csv
Loading ../model_weights/vit_t16_v2/ViT_T16-Derma_SSL_Dino_headless.pth
Number of batches: 1037
Csv file saved: ../datasets/intermediate-features/Fitzpatrick17k-ViT_T16

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading ../model_weights/vit_t16_v1/ViT_T16-ImageNet_1k_SL_WinKawaks_headless.pth
Number of batches: 1037
Csv file saved: ../datasets/intermediate-features/Fitzpatrick17k-ViT_T16-ImageNet_1k_SL_WinKawaks.csv
Loading ../model_weights/vit_t16_v2/ViT_T16-ImageNet_1k_SSL_Dino_headless.pth
Number of batches: 1037
Csv file saved: ../datasets/intermediate-features/Fitzpatrick17k-ViT_T16-ImageNet_1k_SSL_Dino.csv
Loading ../model_weights/vit_t16_v3/ViT_T16-ImageNet_AugReg_headless.pth
Number of batches: 1037
Csv file saved: ../datasets/intermediate-features/Fitzpatrick17k-ViT_T16-ImageNet_AugReg.csv
Loading ../model_weights/vit_t16_v2/ViT_T16-Plant_SSL_Dino_headless.pth
Number of batches: 1037
Csv file saved: ../datasets/intermediate-features/Fitzpatrick17k-ViT_T16-Plant.csv
Loading ../model_weights/vit_t16_v2/ViT_T16-Random_headless.pth
Number of batches: 1037
Csv file saved: ../datasets/intermediate-features/Fitzpatrick17k-ViT_T16-Random.csv
